In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sdv.tabular import CopulaGAN
from sdv.tabular import TVAE
from sdv.tabular import GaussianCopula
from sdv.tabular import CTGAN
from sdv.evaluation import evaluate

from sklearn.preprocessing import PowerTransformer
from sklearn.model_selection import train_test_split

# Function for reshaping data after power transformation
def reshape_after_pt(df_pt, df): 
    df_pt = pd.DataFrame(df_pt, columns=
                             df.columns)
    df_pt.index = df.index
    return df_pt

In [3]:
df = pd.read_csv('allregimes.csv')
df = df[['N','xm','nr','ni','p1','p2','p3']]
df.loc[:, "N"] = np.log2(df['N']).apply(int)
df.sample(5)

,N,xm,nr,ni,p1,p2,p3
257,6,0.437,1.66,0.002,0.271772,0.000089,-0.000695
13,10,0.336,2.00,0.090,3.198936,-0.102964,-0.119347
108,8,0.530,2.00,0.023,1.419347,-0.008927,-0.015329
105,8,0.530,1.45,0.023,1.207729,-0.006966,-0.010055
91,8,0.430,1.68,0.023,0.843626,-0.002038,-0.005224


In [4]:
features = ['N','xm','nr','ni']
response = ['p1','p2','p3']
X = df[features]
y = df[response]

# train/test spliting
# Call train_test_split on X, y. Make the test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2)

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
df_train = X_train.join(y_train)

In [7]:
import os
os.chdir('C:\\Users\\kurt_\\Google Drive\\dell_github\\T-Matrix-Titan\\special\\')

In [7]:
test=[]
best_score = 0.7
for epochs in np.arange(1500, 1600, 50):
        for batch_size in range(40,600,20):
                model = TVAE(
                        epochs=epochs,
                        batch_size=batch_size,
                        field_transformers = {'N':'integer'}
                        )
                model.fit(df_train)
                # let's do a few cross validation for sample evaluation score
                scores = []
                for i in range(20):
                        new_X = model.sample(num_rows=200)
                        scores.append(evaluate(new_X, df_train,metrics=[ 'KSTest']))
                score = np.array(scores).mean()

                if score > best_score:
                        best_score = score
                        best_model = model
                        test.append((epochs, batch_size, score))
                        print(epochs,batch_size,score)

1500 40 0.8652962962962963
1500 120 0.867265873015873
1500 280 0.8681679894179896
1500 360 0.8686375661375662
1500 380 0.874628306878307


In [1]:
samples = np.zeros(50).tolist()
scores=[]
for i in range(50):
        new_X = best_model.sample(num_rows=200)
        samples[i] = new_X
        scores.append(evaluate(new_X, df_train,metrics=[ 'KSTest']))
        print(i, evaluate(new_X, df_train, metrics=[ 'KSTest']))
score = np.array(scores).mean()
score

NameError: name 'np' is not defined

In [11]:
pd.Series(scores).sort_values().tail()

22    0.883757
31    0.884894
10    0.885820
0     0.885952
24    0.886323
dtype: float64

In [21]:
# test_df = pd.DataFrame(test, columns=['epochs','batch_size','score'])
# test_df.sort_values(by='score').tail(55)

Note that We fit a model that is prone to overfitting because we are not testing the new data against a validation set. Instead we are testing the synthetic data with the training data. The reason is by overfitting the model fit we create a data which is too similar too training data, not only the actual trends and relationship is simulated but also the noise is emitated by the overfitted data. I argue that this is better than fitting more generalized model. A more generalized model would 

In [9]:
model = TVAE(
        epochs=1500,
        batch_size=380,
        field_transformers = {'N':'integer'}
        )
model.fit(df_train)
new_df = model.sample(num_rows=200)
evaluate(new_df, df_train) #

0.6928439345611784

In [19]:
samples = np.zeros(50).tolist()
scores=[]
for i in range(50):
        new_df = model.sample(num_rows=200)
        samples[i] = new_df
        scores.append(evaluate(new_df, df_train,metrics=[ 'KSTest']))
        print(i, evaluate(new_df, df_train, metrics=[ 'KSTest']))
score = np.array(scores).mean()
score

In [13]:
pd.Series(scores).sort_values().tail(1)

37    0.882302
dtype: float64

In [21]:
scores2 = []
for i,dt in enumerate(samples):
    scores2.append(evaluate(dt, df_train))
    # print(i, evaluate(dt, df_train))
pd.Series(scores2).sort_values(ascending=False).head()

36    0.718474
48    0.717280
11    0.712742
5     0.711834
31    0.709989
dtype: float64

In [22]:
pd.Series(scores2).sort_values(ascending=False).loc[37]

0.7061150517816707

In [23]:
evaluate(samples[36], df_train, aggregate=False)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,BNLogLikelihood,BayesianNetwork Log Likelihood,NaN,NaN,-inf,0.0,MAXIMIZE,"Cannot find fields of types ('categorical', 'b..."
1,LogisticDetection,LogisticRegression Detection,0.888028,0.888028,0.0,1.0,MAXIMIZE,None
2,SVCDetection,SVC Detection,0.984848,0.984848,0.0,1.0,MAXIMIZE,None
3,BinaryDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
4,BinaryAdaBoostClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
5,BinaryLogisticRegression,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
6,BinaryMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
7,MulticlassDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
8,MulticlassMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
9,LinearRegression,None,NaN,NaN,-inf,1.0,MAXIMIZE,`target` must be passed either directly or ins...


In [24]:
evaluate(samples[37], df_train, aggregate=False)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,BNLogLikelihood,BayesianNetwork Log Likelihood,NaN,NaN,-inf,0.0,MAXIMIZE,"Cannot find fields of types ('categorical', 'b..."
1,LogisticDetection,LogisticRegression Detection,0.941586,0.941586,0.0,1.0,MAXIMIZE,None
2,SVCDetection,SVC Detection,0.959923,0.959923,0.0,1.0,MAXIMIZE,None
3,BinaryDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
4,BinaryAdaBoostClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
5,BinaryLogisticRegression,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
6,BinaryMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
7,MulticlassDecisionTreeClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
8,MulticlassMLPClassifier,None,NaN,NaN,0.0,1.0,MAXIMIZE,`target` must be passed either directly or ins...
9,LinearRegression,None,NaN,NaN,-inf,1.0,MAXIMIZE,`target` must be passed either directly or ins...


This is the best sample. Let's Take that sample and save it to use it in model training

In [18]:
sample[37]

0.8823015873015875

In [35]:

test=[]
best_score = 0.7
for ep_bs in old_test:
    epochs = ep_bs[0]
    batch_size = ep_bs[1]
    for l2scale in (2e-5,3e-5):
        model = TVAE(
                epochs=epochs,
                batch_size=batch_size,
                field_transformers = {'N':'integer'}
                )
        model.fit(df_train)
        new_X = model.sample(num_rows=200)
        # let's do a few cross validation for sample evaluation score
        scores = []
        for i in range(6):
                scores.append(evaluate(new_X, df_train))
        score = np.array(scores).mean()

        if score > best_score:
                best_score = score
                best_model = model
                test.append((epochs, batch_size, l2scale, score))
                print(epochs,batch_size,l2scale, score)

1620 60 2e-05 0.7097647519386623
1620 60 3e-05 0.7193964680545154
